# 1. Data Exploration and Understanding

**Welcome to the Ames Housing Price Prediction Project!**

In this notebook, we will conduct a comprehensive exploratory data analysis (EDA) of the Ames Housing dataset. Think of this as getting to know our data intimately before we build any predictive models.

## What We'll Accomplish

By the end of this notebook, we will have:
- **Understood our target variable** (house prices) and optimized it for modeling
- **Identified the most predictive features** through correlation and statistical analysis
- **Developed a smart strategy for handling missing data** using real estate domain knowledge
- **Detected and analyzed outliers** that could impact our model performance
- **Created a foundation** for feature engineering in our next notebook

## Why This Matters

Exploratory Data Analysis is like being a detective - we're looking for clues about what makes houses expensive or cheap. This investigation will guide all our future decisions about feature engineering, model selection, and validation strategies.

## 1.1 Environment Setup

First, let's import all the libraries we'll need for our analysis. Each library serves a specific purpose in our data exploration journey.

In [ ]:
# Data manipulation and numerical operations
import pandas as pd              # For working with structured data (like spreadsheets)
import numpy as np               # For mathematical operations and array handling

# Visualization libraries
import matplotlib.pyplot as plt  # For creating basic plots and charts
import seaborn as sns            # For beautiful statistical visualizations

# Statistical analysis
from scipy import stats          # For statistical tests and distributions
from scipy.stats import skew, kurtosis, shapiro  # For measuring data distribution characteristics

# Machine learning utilities
from sklearn.feature_selection import mutual_info_regression  # For detecting non-linear relationships

# Configuration for better output
import warnings
warnings.filterwarnings('ignore')  # Hide warning messages for cleaner output

# Set up pandas to show more data
pd.set_option('display.max_columns', None)  # Show all columns when displaying dataframes
pd.set_option('display.max_rows', 20)       # Limit rows to keep output manageable

# Configure matplotlib for better-looking plots
plt.style.use('seaborn-v0_8-whitegrid')     # Use a clean, professional style
sns.set_palette("husl")                     # Use a colorful, distinguishable palette
plt.rcParams['figure.figsize'] = (12, 8)    # Set default figure size
plt.rcParams['font.size'] = 11              # Set default font size

print("✓ All libraries imported successfully!")
print("✓ Visualization settings configured for professional output")

**Why these specific libraries?**

- **Pandas & NumPy**: The foundation of data science in Python - they handle all our data manipulation needs
- **Matplotlib & Seaborn**: Create beautiful, informative visualizations that help us understand patterns
- **SciPy**: Provides advanced statistical functions for testing hypotheses about our data
- **Scikit-learn**: Even for EDA, we use some ML utilities to understand feature relationships

Now let's load our dataset and take our first look at the Ames housing data.

## 1.2 Data Loading and First Impressions

Let's load our dataset and get our first look at what we're working with. The Ames Housing dataset contains information about residential properties sold in Ames, Iowa between 2006-2010.

In [ ]:
# Load the Ames Housing dataset
try:
    df = pd.read_csv("../data/raw/train.csv")
    print(f"✓ Dataset loaded successfully from: ../data/raw/train.csv")
    print(f"✓ Dataset shape: {df.shape[0]:,} houses with {df.shape[1]} features each")
except FileNotFoundError:
    print("❌ ERROR: Dataset file not found. Please ensure ../data/raw/train.csv exists.")
    raise

# Get basic information about our dataset
print(f"\n=== DATASET OVERVIEW ===")
print(f"Total properties: {df.shape[0]:,}")
print(f"Total features: {df.shape[1]}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
print(f"Time period: {df['YrSold'].min()}-{df['YrSold'].max()}")
print(f"Duplicate rows: {df.duplicated().sum()}")

**What we just learned:**

This gives us our first overview of the dataset size and scope. With nearly 1,500 houses and 81 features, we have a rich dataset for building predictive models. The 5-year time span (2006-2010) includes both pre- and post-financial crisis data, which could be important for our analysis.

Let's examine the structure of our data more closely:

In [ ]:
# Feature type analysis - Understanding what kinds of data we have
numerical_features = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = df.select_dtypes(include=['object']).columns.tolist()

# Remove Id and target from numerical features for analysis
if 'Id' in numerical_features: 
    numerical_features.remove('Id')
if 'SalePrice' in numerical_features: 
    numerical_features.remove('SalePrice')

print(f"=== FEATURE TYPE BREAKDOWN ===")
print(f"Numerical features: {len(numerical_features)} (continuous/discrete numbers)")
print(f"Categorical features: {len(categorical_features)} (text categories)")
print(f"Target variable: SalePrice (what we want to predict)")
print(f"Identifier: Id (unique house identifier)")

# Display basic statistics
print(f"\n=== BASIC STATISTICAL SUMMARY ===")
print("Target Variable (SalePrice) Statistics:")
price_stats = df['SalePrice'].describe()
for stat, value in price_stats.items():
    if stat == 'count':
        print(f"  {stat}: {value:,.0f}")
    else:
        print(f"  {stat}: ${value:,.0f}")

# Show a sample of our data with key features
print(f"\n=== SAMPLE DATA (First 3 Houses) ===")
key_columns = ['Id', 'MSSubClass', 'MSZoning', 'LotArea', 'OverallQual', 'OverallCond', 
               'YearBuilt', 'GrLivArea', 'BedroomAbvGr', 'TotRmsAbvGrd', 'SalePrice']
sample_data = df[key_columns].head(3)
print(sample_data.to_string(index=False))

In [ ]:
# Perform statistical tests for normality
# Note: Shapiro-Wilk test has limitations for large samples, so we'll use a subset
sample_size = 500  # Use a representative sample for the test
np.random.seed(42)  # For reproducible results

# Create random samples from both distributions
original_sample = np.random.choice(prices, size=sample_size, replace=False)
log_sample = np.random.choice(log_prices, size=sample_size, replace=False)

# Perform Shapiro-Wilk tests
original_shapiro_stat, original_shapiro_p = shapiro(original_sample)
log_shapiro_stat, log_shapiro_p = shapiro(log_sample)

print("=== STATISTICAL NORMALITY TESTS ===")
print("Shapiro-Wilk Test Results (Higher statistic = more normal, p > 0.05 = normal)")
print("-" * 70)
print(f"Original Prices:")
print(f"  Test Statistic: {original_shapiro_stat:.6f}")
print(f"  P-value: {original_shapiro_p:.2e}")
print(f"  Conclusion: {'Normal distribution' if original_shapiro_p > 0.05 else 'Not normal distribution'}")

print(f"\nLog-Transformed Prices:")
print(f"  Test Statistic: {log_shapiro_stat:.6f}")
print(f"  P-value: {log_shapiro_p:.2e}")
print(f"  Conclusion: {'Normal distribution' if log_shapiro_p > 0.05 else 'Not normal distribution'}")

# Calculate improvement metrics
stat_improvement = ((log_shapiro_stat - original_shapiro_stat) / original_shapiro_stat) * 100
p_value_ratio = log_shapiro_p / original_shapiro_p if original_shapiro_p > 0 else float('inf')

print(f"\n=== STATISTICAL IMPROVEMENT ANALYSIS ===")
print(f"Test Statistic Improvement: {stat_improvement:.2f}%")
print(f"P-value Improvement Ratio: {p_value_ratio:.1f}x better")

# Create a comprehensive summary of all improvements
print(f"\n=== COMPREHENSIVE TRANSFORMATION SUMMARY ===")
print(f"✓ Skewness: {original_stats['skewness']:.3f} → {transformed_stats['skewness']:.3f} ({skewness_improvement:.1f}% better)")
print(f"✓ Kurtosis: {original_stats['kurtosis']:.3f} → {transformed_stats['kurtosis']:.3f} ({kurtosis_improvement:.1f}% better)")
print(f"✓ Shapiro-Wilk: {original_shapiro_stat:.4f} → {log_shapiro_stat:.4f} ({stat_improvement:.1f}% better)")
print(f"✓ Distribution Shape: {'Right-skewed' if original_stats['skewness'] > 0.5 else 'Normal'} → {'Nearly Normal' if abs(transformed_stats['skewness']) < 0.5 else 'Improved'}")

# Final recommendation
recommendation = "STRONGLY RECOMMENDED" if (abs(transformed_stats['skewness']) < abs(original_stats['skewness']) and 
                                             log_shapiro_stat > original_shapiro_stat) else "RECOMMENDED"

print(f"\n=== FINAL RECOMMENDATION ===")
print(f"Log Transformation Status: {recommendation}")
print(f"Reasoning:")
print(f"  • Significantly reduced skewness for better model performance")
print(f"  • Improved statistical normality (higher Shapiro-Wilk statistic)")
print(f"  • Better distribution shape for linear regression assumptions")
print(f"  • Reduced impact of extreme outliers")
print(f"\n✓ We will use log-transformed prices (LogSalePrice) as our target variable for all modeling")

**Transformation Results Analysis:**

The log transformation has dramatically improved our target variable distribution:

1. **Skewness Reduction**: The transformation significantly reduced skewness, making the distribution much more symmetric
2. **Better Normality**: The Q-Q plots show the log-transformed prices follow a normal distribution much more closely
3. **Reduced Outlier Impact**: The box plots show fewer extreme outliers in the transformed data
4. **Modeling Benefits**: This transformation will help our regression models perform better and make more accurate predictions

## 2.3 Statistical Validation of Transformation

Let's formally test whether our log transformation has improved the normality of our target variable using statistical tests.

In [ ]:
# Apply log transformation to house prices
# We use log1p (log(1 + x)) to handle any zero values safely
log_prices = np.log1p(prices)

# Calculate statistics for both original and transformed prices
original_stats = {
    'mean': prices.mean(),
    'median': prices.median(),
    'std': prices.std(),
    'skewness': skew(prices),
    'kurtosis': kurtosis(prices)
}

transformed_stats = {
    'mean': log_prices.mean(),
    'median': log_prices.median(),
    'std': log_prices.std(),
    'skewness': skew(log_prices),
    'kurtosis': kurtosis(log_prices)
}

print("=== TRANSFORMATION COMPARISON ===")
print(f"{'Metric':<12} {'Original':<15} {'Log-Transformed':<15} {'Improvement'}")
print("-" * 60)
print(f"{'Mean':<12} ${original_stats['mean']:<14,.0f} {transformed_stats['mean']:<15.3f} {'N/A'}")
print(f"{'Median':<12} ${original_stats['median']:<14,.0f} {transformed_stats['median']:<15.3f} {'N/A'}")
print(f"{'Std Dev':<12} ${original_stats['std']:<14,.0f} {transformed_stats['std']:<15.3f} {'N/A'}")
print(f"{'Skewness':<12} {original_stats['skewness']:<15.3f} {transformed_stats['skewness']:<15.3f} {abs(original_stats['skewness']) - abs(transformed_stats['skewness']):.3f}")
print(f"{'Kurtosis':<12} {original_stats['kurtosis']:<15.3f} {transformed_stats['kurtosis']:<15.3f} {abs(original_stats['kurtosis']) - abs(transformed_stats['kurtosis']):.3f}")

# Create side-by-side comparison visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Price Distribution: Original vs Log-Transformed', fontsize=16, fontweight='bold')

# Original distribution plots
axes[0, 0].hist(prices, bins=50, alpha=0.7, color='lightcoral', edgecolor='black')
axes[0, 0].set_title('Original Price Distribution')
axes[0, 0].set_xlabel('Sale Price ($)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].boxplot(prices, vert=True, patch_artist=True, 
                   boxprops=dict(facecolor='lightcoral'))
axes[0, 1].set_title('Original Price Box Plot')
axes[0, 1].set_ylabel('Sale Price ($)')
axes[0, 1].grid(True, alpha=0.3)

stats.probplot(prices, dist="norm", plot=axes[0, 2])
axes[0, 2].set_title('Original Q-Q Plot')
axes[0, 2].grid(True, alpha=0.3)

# Log-transformed distribution plots
axes[1, 0].hist(log_prices, bins=50, alpha=0.7, color='lightgreen', edgecolor='black')
axes[1, 0].set_title('Log-Transformed Price Distribution')
axes[1, 0].set_xlabel('Log(Sale Price)')
axes[1, 0].set_ylabel('Frequency')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].boxplot(log_prices, vert=True, patch_artist=True, 
                   boxprops=dict(facecolor='lightgreen'))
axes[1, 1].set_title('Log-Transformed Price Box Plot')
axes[1, 1].set_ylabel('Log(Sale Price)')
axes[1, 1].grid(True, alpha=0.3)

stats.probplot(log_prices, dist="norm", plot=axes[1, 2])
axes[1, 2].set_title('Log-Transformed Q-Q Plot')
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate improvement metrics
skewness_improvement = ((abs(original_stats['skewness']) - abs(transformed_stats['skewness'])) / abs(original_stats['skewness'])) * 100
kurtosis_improvement = ((abs(original_stats['kurtosis']) - abs(transformed_stats['kurtosis'])) / abs(original_stats['kurtosis'])) * 100

print(f"\n=== TRANSFORMATION EFFECTIVENESS ===")
print(f"Skewness reduction: {skewness_improvement:.1f}% improvement")
print(f"Kurtosis reduction: {kurtosis_improvement:.1f}% improvement")
print(f"Distribution shape: {'Much more normal' if abs(transformed_stats['skewness']) < 0.5 else 'Somewhat improved'}")

# Store the transformed target variable for future use
df['LogSalePrice'] = log_prices
print(f"\n✓ Log-transformed target variable created as 'LogSalePrice' column")
print(f"✓ Ready to use log-transformed prices for modeling")

**Key Insights from Price Distribution Analysis:**

1. **Right-Skewed Distribution**: The price distribution shows clear right skewness, meaning there are more affordable houses with a few expensive outliers pulling the mean higher than the median.

2. **Market Reality**: The $7,000+ difference between mean and median reflects typical real estate markets where luxury properties drive up averages.

3. **Outlier Presence**: The box plot and outlier analysis reveal high-end properties that could either be luxury homes or potential data errors.

4. **Non-Normal Distribution**: The Q-Q plot shows our prices don't follow a normal distribution, which can be problematic for linear regression models.

**Why This Matters for Modeling:**
- Most machine learning algorithms perform better with normally distributed target variables
- The right skew could cause our model to be biased toward predicting higher prices
- We should consider transforming our target variable to improve model performance

## 2.2 Target Variable Transformation

Based on our distribution analysis, let's apply a log transformation to normalize the price distribution. This is a common and effective technique in real estate price prediction.

In [ ]:
# Calculate comprehensive statistics for house prices
prices = df['SalePrice']

# Calculate distribution statistics
mean_price = prices.mean()
median_price = prices.median()
std_price = prices.std()
skewness = skew(prices)
kurt = kurtosis(prices)
min_price = prices.min()
max_price = prices.max()
q1, q3 = prices.quantile([0.25, 0.75])
iqr = q3 - q1

print("=== COMPREHENSIVE PRICE DISTRIBUTION ANALYSIS ===")
print(f"Mean price: ${mean_price:,.0f}")
print(f"Median price: ${median_price:,.0f}")
print(f"Standard deviation: ${std_price:,.0f}")
print(f"Price range: ${min_price:,.0f} - ${max_price:,.0f}")
print(f"Interquartile range (Q1-Q3): ${q1:,.0f} - ${q3:,.0f}")
print(f"\n=== DISTRIBUTION CHARACTERISTICS ===")
print(f"Skewness: {skewness:.3f} ({'Right-skewed' if skewness > 0.5 else 'Nearly symmetric' if abs(skewness) <= 0.5 else 'Left-skewed'})")
print(f"Kurtosis: {kurt:.3f} ({'Heavy-tailed' if kurt > 3 else 'Light-tailed' if kurt < 3 else 'Normal-tailed'})")
print(f"Mean vs Median difference: ${mean_price - median_price:,.0f} ({((mean_price - median_price)/median_price)*100:.1f}%)")

# Create comprehensive visualization of price distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('House Price Distribution Analysis', fontsize=16, fontweight='bold')

# 1. Histogram with statistics
ax1 = axes[0, 0]
ax1.hist(prices, bins=50, alpha=0.7, color='skyblue', edgecolor='black')
ax1.axvline(mean_price, color='red', linestyle='--', linewidth=2, label=f'Mean: ${mean_price:,.0f}')
ax1.axvline(median_price, color='green', linestyle='--', linewidth=2, label=f'Median: ${median_price:,.0f}')
ax1.set_xlabel('Sale Price ($)')
ax1.set_ylabel('Number of Houses')
ax1.set_title('Price Distribution with Central Tendencies')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Box plot showing outliers
ax2 = axes[0, 1]
box_plot = ax2.boxplot(prices, vert=True, patch_artist=True)
box_plot['boxes'][0].set_facecolor('lightcoral')
ax2.set_ylabel('Sale Price ($)')
ax2.set_title('Box Plot - Outlier Detection')
ax2.grid(True, alpha=0.3)
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1000:.0f}K'))

# 3. Q-Q plot against normal distribution
ax3 = axes[1, 0]
stats.probplot(prices, dist="norm", plot=ax3)
ax3.set_title('Q-Q Plot vs Normal Distribution')
ax3.grid(True, alpha=0.3)

# 4. Density plot with normal overlay
ax4 = axes[1, 1]
ax4.hist(prices, bins=50, density=True, alpha=0.7, color='lightblue', label='Actual Distribution')
# Overlay normal distribution for comparison
x_norm = np.linspace(prices.min(), prices.max(), 100)
y_norm = stats.norm.pdf(x_norm, mean_price, std_price)
ax4.plot(x_norm, y_norm, 'r-', linewidth=2, label='Normal Distribution')
ax4.set_xlabel('Sale Price ($)')
ax4.set_ylabel('Density')
ax4.set_title('Density Plot with Normal Comparison')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Identify outliers using IQR method
outlier_threshold_low = q1 - 1.5 * iqr
outlier_threshold_high = q3 + 1.5 * iqr
outliers = prices[(prices < outlier_threshold_low) | (prices > outlier_threshold_high)]

print(f"\n=== OUTLIER ANALYSIS ===")
print(f"Outlier thresholds: ${outlier_threshold_low:,.0f} - ${outlier_threshold_high:,.0f}")
print(f"Number of outliers: {len(outliers)} ({len(outliers)/len(prices)*100:.1f}% of data)")
if len(outliers) > 0:
    print(f"Outlier price range: ${outliers.min():,.0f} - ${outliers.max():,.0f}")

**Understanding Our Data Structure:**

From this analysis, we can see that our dataset is well-balanced with both numerical and categorical features. The sample data shows us houses with different characteristics:
- **Variety in size**: From 1,710 to 2,198 square feet of living area
- **Quality differences**: Overall quality ratings from 5 to 8 (out of 10)
- **Age variation**: Houses built between 1961-2003  
- **Price range**: From $109,500 to $215,000

This diversity suggests our model will have plenty of variation to learn from. Now let's dive deep into our target variable - the house prices we want to predict.

---

# 2. Target Variable Analysis

Understanding our target variable (SalePrice) is crucial because it influences how we build our model. We need to understand its distribution, detect any issues, and potentially transform it for better modeling performance.

## 2.1 Price Distribution Analysis

Let's start by examining how house prices are distributed in our dataset. This will help us understand the market dynamics and identify any characteristics that might affect our modeling approach.